In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms

# transformations for the test dataset
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# load CIFAR-10 test dataset
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test
)

# CIFAR-10 class names
class_names = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

# define a function to unnormalize and convert the image back to a NumPy array
def unnormalize_image(image_tensor):
    mean = np.array([0.4914, 0.4822, 0.4465])
    std = np.array([0.2023, 0.1994, 0.2010])
    image = image_tensor.numpy().transpose((1, 2, 0))  # convert from (C, H, W) to (H, W, C)
    image = std * image + mean  # unnormalize
    image = np.clip(image, 0, 1)  # clip to valid range [0, 1]
    return image

# plot image with its label distribution
def plot_image(image_tensor, label):
    # unnormalize and convert image to NumPy format
    image = unnormalize_image(image_tensor)
    
    # plot image and label distribution
    plt.figure(figsize=(6, 3))

    # image
    plt.imshow(image)
    plt.axis('off')
    plt.title(class_names[label])

    plt.show()

# show examples for specific indices
for i in range(2): 
    image_tensor, label = testset[i]
    plot_image(image_tensor,label)

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
from models import *
import torch.backends.cudnn as cudnn

# model
net = EfficientNetB0()

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

weights_path = "weights/EfficientNetB0_0.1_100_512_SGD_1" 

# load weights into the model
net.load_state_dict(torch.load(weights_path))
print("Model weights loaded successfully!")

In [ ]:
criterion = nn.CrossEntropyLoss() 

net.eval()

In [ ]:
testset_size = len(testset)
calibration_size = 1000
final_test_size = len(testset) - calibration_size

In [ ]:
# binary search parameters
tolerance = 0.005
epsilon = 1e-10

In [ ]:
# function used to compute alpha_loo
def compute_alpha_i(C, calibration_logits, calibration_scores, i):

    sum_label = sum(calibration_scores[j] for j in range(len(calibration_scores)) if j != i)

    rs = []
    with torch.no_grad():
        for k in range(10):
            true_label_tensor_random = torch.tensor([k], dtype=torch.long).to(device)
            S = criterion(calibration_logits[i], true_label_tensor_random).item()
            ratio = (calibration_size + 1) * S / (sum_label + S)
            rs.append(ratio)

    left, right = epsilon, 1.0 - epsilon
    while right - left > tolerance:
        alpha = (left + right) / 2
        threshold = 1 / alpha

        conformal_set = [k for k, ratio in enumerate(rs) if ratio < threshold]

        if len(conformal_set) <= C:
            right = alpha
        else:
            left = alpha

    alpha_min = (left + right) / 2
    return alpha_min

In [ ]:
from torchvision.models import resnet18
from torch import nn

# load pretrained ResNet18
extractor = resnet18(pretrained=True)
extractor.fc = nn.Identity()  # remove final classification layer
extractor.eval()

# function to extract features using ResNet18
def extract_features(X):
    features = []
    with torch.no_grad():
        for x in X:
            feat = extractor(x.unsqueeze(0)) 
            features.append(feat)
    return torch.cat(features, dim=0).numpy()

In [ ]:
def map_entropies_to_discrete_sizes(entropies, min_entropy, max_entropy, max_size=3, min_size=1):
    num_bins = max_size - min_size + 1
    scaled = np.linspace(0, 1, num_bins) ** (1/2)
    bins = min_entropy + (max_entropy - min_entropy) * scaled
    return np.digitize(entropies, bins, right=True) + min_size

In [ ]:
from torch.utils.data import random_split
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from scipy.stats import entropy

# for iteration
proba = 0
alphas_mins = [] # store the true alpha
alphas_loos = [] # store the leave-one-out estimates of alpha
Niter = 200
set_sizes = []

for iter in range(Niter):
    print(f"Iteration {iter}")

    # split the test set
    calibration_set, final_test_set = random_split(
        testset,
        [calibration_size, final_test_size],
        generator=torch.Generator().manual_seed(iter)
    )

    calibration_scores = [] # store the scores
    calibration_logits = [] # store the logits (features X)

    # compute calibration scores
    with torch.no_grad():
        for idx in range(calibration_size):
            x_sample, y_true = calibration_set[idx]
            x_sample = x_sample.unsqueeze(0).to(device)
            y_true = torch.tensor([y_true], dtype=torch.long).to(device)
            logits = net(x_sample)
            score = criterion(logits, y_true).item()
            calibration_logits.append(logits)
            calibration_scores.append(score)
    sum_label = sum(calibration_scores)

    # sample one random element from the final test set
    random_idx = np.random.choice(final_test_size) 
    x_random, y_random = final_test_set[random_idx]
    y_random = int(y_random) 
    true_label = y_random

    x_random_tensor = x_random.unsqueeze(0).to(device)
    logits_random = net(x_random_tensor)

    model_prediction = torch.argmax(logits_random).item()

    X_calib = torch.stack([x[0] for x in calibration_set])
    y_calib = torch.tensor([x[1] for x in calibration_set])

    X_calib_features = extract_features(X_calib)

    pca = PCA(n_components=2)
    X_calib_pca = pca.fit_transform(X_calib_features)
    x_random_features = extract_features(x_random_tensor.cpu())
    x_random_sample_pca = pca.transform(x_random_features)  
    
    # k-NN
    k = 20
    nn = NearestNeighbors(n_neighbors=k + 1).fit(X_calib_pca)

    # local entropies
    local_entropies_calib = []
    for x in X_calib_pca:
        distances, indices = nn.kneighbors([x])
        neighbor_labels = y_calib[indices[0][1:]]
        counts = np.bincount(neighbor_labels, minlength=10)
        probs = counts / counts.sum()
        local_entropies_calib.append(entropy(probs, base=2))
    local_entropies_calib = np.array(local_entropies_calib)

    # local entropy around test sample
    distances, indices = nn.kneighbors(x_random_sample_pca)
    neighbor_labels = y_calib[indices[0]]
    counts = np.bincount(neighbor_labels, minlength=10)
    probs = counts / counts.sum()
    local_entropy_test = entropy(probs, base=2)

    # print(f"Random test sample index: {idx}, label: {true_label}")
    # print(f"Local label entropy (k={k}) around test point: {local_entropy_test:.4f} bits")

    min_entropy = np.min(local_entropies_calib)
    max_entropy = np.max(local_entropies_calib)
    set_size = map_entropies_to_discrete_sizes(local_entropy_test,min_entropy,max_entropy)
    # print(f"Set size = {set_size}")
    set_sizes.append(set_size)

    # compute ratio vector
    rs = []
    for k in range(10): 
        true_label_tensor_random = torch.tensor([k], dtype=torch.long).to(device)
        S = criterion(logits_random, true_label_tensor_random).item()
        ratio = (calibration_size + 1) * S / (sum_label + S)
        rs.append(ratio)

    # compute alpha_min (\tilde{\alpha}) using binary search
    left, right = epsilon, 1.0 - epsilon
    while right - left > tolerance:
        alpha = (left + right) / 2
        threshold = 1 / alpha

        conformal_set = [k for k, ratio in enumerate(rs) if ratio < threshold]

        if len(conformal_set) <= set_size:
            right = alpha  # try smaller alpha
        else:
            left = alpha   # need larger alpha

    alpha_min = (left + right) / 2
    alphas_mins.append(alpha_min)
    
    # update empirical coverage probability
    if true_label in conformal_set:
        proba += 1

    print("Computing LOO estimator...")
    list_alpha_i = []
    for i in range(len(calibration_logits)):
        if i%10==0:
            print(i)
        list_alpha_i.append(compute_alpha_i(set_size,calibration_logits,calibration_scores,i))
    alpha_loo = sum(list_alpha_i)/len(calibration_logits)
    alphas_loos.append(alpha_loo)

        
proba = proba/Niter
alpha_mean = sum(alphas_mins)/Niter 
alpha_loo_mean = sum(alphas_loos)/Niter
print(f"Proba = {proba}")
print(f"1-E[alpha] = {1-alpha_mean}")
print(f"E[alpha] = {alpha_mean}")
print(f"E[alpha_LOO] = {alpha_loo_mean}")

In [ ]:
# plot histogram of 1-alpha
alphas = np.arange(0.005,0.505,0.01)

fig_alpha, ax_alpha = plt.subplots(figsize=(7, 4)) 
ax_alpha.hist(1-np.array(alphas_mins), color='green',alpha=0.4,bins=1-np.array(alphas[::-1]),align='left',label=r"$\tilde{\alpha}$")
ax_alpha.hist(1-np.array(alphas_loos), color='blue',alpha=0.4,bins=1-np.array(alphas[::-1]),align='left',label=r"$\hat{\alpha}^{LOO}$")
ax_alpha.axvline(x=proba, color='red', linestyle='--', linewidth=3,label=r"$Pr(Y_{\rm test} \in \hat{C}_n(X_{\rm test}))$")
ax_alpha.axvline(x=np.mean(1-np.array(alphas_mins)), color='green', linestyle='--', linewidth=3)
ax_alpha.axvline(x=np.mean(1-np.array(alphas_loos)), color='blue', linestyle='--', linewidth=3)
ax_alpha.set_xlabel(r"$1-\tilde{\alpha}$",fontsize=16)
ax_alpha.set_ylabel("Frequency",fontsize=16)

ax_alpha.set_xlim(0.8,1)
ax_alpha.set_ylim(0,105)

ax_alpha.tick_params(axis='both', labelsize=14)

ax_alpha.set_xticks(np.linspace(0.5, 1.0, 11))
ax_alpha.set_xticks(np.linspace(0.5, 1.0, 21),minor=True)

ax_alpha.grid(True, linestyle="--", alpha=0.4)

plt.legend(fontsize=14)
plt.tight_layout()
plt.savefig("plots/hist_n"+str(calibration_size)+".pdf", format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
np.save("alphas/alphas_mins_n"+str(calibration_size)+".npy", alphas_mins)
np.save("alphas/alphas_loos_n"+str(calibration_size)+".npy", alphas_loos)